In [15]:
# Superdense coding demo (Qiskit >= 1.0, no primitives)

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt   # for inline drawing
backend = AerSimulator()

# ----- 1. Choose the 2-bit classical message -----
message = "10"   # try: "00", "01", "10", "11"

# ----- 2. Build the superdense coding circuit -----
qc = QuantumCircuit(2, 2)   # q0 = Alice, q1 = Bob

# (a) Create shared Bell pair between Alice and Bob
qc.h(0)
qc.cx(0, 1)

# (b) Alice encodes her 2 classical bits on her qubit (q0)
if message == "00":
    pass          # do nothing = I
elif message == "01":
    qc.x(0)       # X
elif message == "10":
    qc.z(0)       # Z
elif message == "11":
    qc.x(0); qc.z(0)  # XZ

# (Alice now sends her qubit q0 to Bob)

# (c) Bob decodes with a Bell-basis measurement
qc.cx(0, 1)
qc.h(0)
qc.measure([0, 1], [0, 1])   # measure into classical bits c0,c1

print(qc.draw())  # show the circuit

# ----- 3. Run on simulator and look at the result -----
tqc = transpile(qc, backend)
result = backend.run(tqc, shots=1024).result()
counts = result.get_counts()

# after running the circuit and getting counts
raw_counts = counts
decoded_counts = {bits[::-1]: c for bits, c in raw_counts.items()}

print("Target message:", message)
print("Raw counts (Qiskit order):", raw_counts)
print("Decoded counts (b1b0):", decoded_counts)



     ┌───┐     ┌───┐     ┌───┐┌─┐
q_0: ┤ H ├──■──┤ Z ├──■──┤ H ├┤M├
     └───┘┌─┴─┐└───┘┌─┴─┐└┬─┬┘└╥┘
q_1: ─────┤ X ├─────┤ X ├─┤M├──╫─
          └───┘     └───┘ └╥┘  ║ 
c: 2/══════════════════════╩═══╩═
                           1   0 
Target message: 10
Raw counts (Qiskit order): {'01': 1024}
Decoded counts (b1b0): {'10': 1024}


Then you can add some noise and see how the system behave on all the messages and see the output.

Go through line by line and try to understand the flow.

In [5]:
# Qiskit >= 1.0
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

def superdense_circuit(msg: str) -> QuantumCircuit:
    """
    q0 = Alice's qubit, q1 = Bob's qubit.
    msg in {'00','01','10','11'} mapped to I, X, Z, XZ on Alice's qubit.
    """
    qc = QuantumCircuit(2, 2)

    # 1) Share Bell pair
    qc.h(0)
    qc.cx(0, 1)

    # 2) Alice encodes 2 classical bits onto her qubit
    if msg == '00':
        pass
    elif msg == '01':
        qc.x(0)
    elif msg == '10':
        qc.z(0)
    elif msg == '11':
        qc.x(0); qc.z(0)
    else:
        raise ValueError("msg must be one of '00','01','10','11'")

    # (Alice sends her qubit to Bob)

    # 3) Bob decodes via Bell-basis measurement
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0, 1], [0, 1])

    return qc

# --- Noise model (same spirit as your teleportation example) ---
noise = NoiseModel()
noise.add_all_qubit_quantum_error(depolarizing_error(0.002, 1), ['h','x','y','z','sx'])
noise.add_all_qubit_quantum_error(depolarizing_error(0.01,  2), ['cx'])

backend = AerSimulator(noise_model=noise)
shots = 2000

def decode_accuracy(msg: str) -> float:
    qc = superdense_circuit(msg)
    tqc = transpile(qc, backend)
    result = backend.run(tqc, shots=shots).result()
    counts = result.get_counts()
    # Measurement bitstring comes out as 'b1b0' from c1,c0
    # We built measure([0,1],[0,1]) so key is c1c0 or c0c1 depending on backend order.
    # With our mapping, the classical bits map as: c0 <- q0, c1 <- q1.
    # Bob’s Bell measurement yields exactly the 2-bit message as the readout 'b1b0' = msg.
    success = counts.get(msg, 0)
    return success / shots, counts

for m in ['00','01','10','11']:
    acc, counts = decode_accuracy(m)
    print(f"Message {m} → accuracy ≈ {acc:.3f}, counts: {counts}")


Message 00 → accuracy ≈ 1.000, counts: {'00': 2000}
Message 01 → accuracy ≈ 0.000, counts: {'10': 1998, '00': 2}
Message 10 → accuracy ≈ 0.000, counts: {'01': 1998, '00': 2}
Message 11 → accuracy ≈ 0.998, counts: {'11': 1995, '01': 4, '10': 1}
